In [2]:
import requests

def get_latest_cvss_score(cve_id):
    # Define the NVD API URL
    nvd_api_url = f"https://services.nvd.nist.gov/rest/json/cves/2.0?cveId={cve_id}"
    
    # Make the request to the NVD API
    response = requests.get(nvd_api_url)
    
    if response.status_code != 200:
        raise Exception(f"Failed to fetch data for {cve_id}. HTTP Status Code: {response.status_code}")
    
    # Parse the JSON response
    data = response.json()
    
    # Check if vulnerabilities data is present
    if not data.get("vulnerabilities"):
        raise Exception(f"No vulnerabilities data found for {cve_id}")
    
    vulnerability_data = data["vulnerabilities"][0]["cve"]["metrics"]
    
    # Check for available CVSS metrics and choose the highest version available
    if "cvssMetricV31" in vulnerability_data:
        latest_cvss = vulnerability_data["cvssMetricV31"][0]["cvssData"]
    elif "cvssMetricV30" in vulnerability_data:
        latest_cvss = vulnerability_data["cvssMetricV30"][0]["cvssData"]
    elif "cvssMetricV2" in vulnerability_data:
        latest_cvss = vulnerability_data["cvssMetricV2"][0]["cvssData"]
    else:
        raise Exception(f"No CVSS data available for {cve_id}")
    
    # Extract the base score and severity
    base_score = latest_cvss["baseScore"]
    base_severity = latest_cvss["baseSeverity"]

    return {
        "cve_id": cve_id,
        "cvss_version": latest_cvss["version"],
        "base_score": base_score,
        "base_severity": base_severity
    }

# Example usage
if __name__ == "__main__":
    cve_id = "CVE-2024-39717"
    result = get_latest_cvss_score(cve_id)
    print(result)

{'cve_id': 'CVE-2024-39717', 'cvss_version': '3.1', 'base_score': 7.2, 'base_severity': 'HIGH'}


In [3]:
import requests
import time
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import logging
from typing import Dict, Any

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

API_URL = "https://services.nvd.nist.gov/rest/json/cves/2.0"

def get_nvd_data(cve_id: str) -> Dict[str, Any]:
    """Fetches available CVSS scores and severities from NVD API.

    Args:
        cve_id: The CVE ID to query.

    Returns:
        A dictionary with CVSS version, score, and severity.
    """
    url = f"{API_URL}?cveId={cve_id}"
    retries = 3  # Number of retries in case of rate limit errors
    backoff_time = 30  # Initial backoff time in seconds for 403 errors

    for attempt in range(retries):
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()

            if "vulnerabilities" not in data or not data["vulnerabilities"]:
                raise Exception(f"No vulnerabilities data found for {cve_id}")

            vulnerability_data = data["vulnerabilities"][0]["cve"]["metrics"]

            # Check for available CVSS metrics and choose the highest version available
            if "cvssMetricV31" in vulnerability_data:
                latest_cvss = vulnerability_data["cvssMetricV31"][0]["cvssData"]
            elif "cvssMetricV30" in vulnerability_data:
                latest_cvss = vulnerability_data["cvssMetricV30"][0]["cvssData"]
            elif "cvssMetricV2" in vulnerability_data:
                latest_cvss = vulnerability_data["cvssMetricV2"][0]["cvssData"]
            else:
                raise Exception(f"No CVSS data available for {cve_id}")

            # Extract the base score and severity, with checks
            base_score = latest_cvss.get("baseScore")
            base_severity = latest_cvss.get("baseSeverity")

            return {
                "cvss_version": latest_cvss["version"],
                "base_score": base_score,
                "base_severity": base_severity
            }

        except requests.exceptions.HTTPError as e:
            if response.status_code == 403:
                logging.error(f"Rate limit exceeded for CVE {cve_id}. Waiting {backoff_time} seconds before retrying...")
                time.sleep(backoff_time)
                backoff_time *= 2  # Exponential backoff
            else:
                logging.error(f"Failed to fetch data for CVE {cve_id}: {e}")
                return {
                    "cvss_version": None,
                    "base_score": None,
                    "base_severity": None
                }
        except requests.exceptions.RequestException as e:
            logging.error(f"Failed to fetch data for CVE {cve_id}: {e}")
            return {
                "cvss_version": None,
                "base_score": None,
                "base_severity": None
            }
        except Exception as e:
            logging.error(f"Failed to process data for CVE {cve_id}: {e}")
            return {
                "cvss_version": None,
                "base_score": None,
                "base_severity": None
            }

    logging.error(f"Exceeded maximum retries for CVE {cve_id}.")
    return {
        "cvss_version": None,
        "base_score": None,
        "base_severity": None
    }

def enrich_parquet_with_nvd_data(parquet_file: str, output_file: str) -> None:
    """Enriches Parquet file with CVSS version, score, and severity from NVD.

    Args:
        parquet_file: Path to the input Parquet file.
        output_file: Path to the output Parquet file with enriched data.
    """
    # Load the existing Parquet file
    df = pd.read_parquet(parquet_file, engine='pyarrow')

    # Initialize lists to store new columns
    cvss_versions = []
    base_scores = []
    base_severities = []

    # Iterate through each row to enrich data
    for index, row in df.iterrows():
        cve_id = row.get("cve")
        if cve_id:
            logging.info(f"Processing CVE {cve_id} (index {index})")
            nvd_data = get_nvd_data(cve_id)
            cvss_versions.append(nvd_data["cvss_version"])
            base_scores.append(nvd_data["base_score"])
            base_severities.append(nvd_data["base_severity"])

            # Print out the results
            print(f"CVE: {cve_id}")
            print(f"CVSS Version: {nvd_data['cvss_version']}")
            print(f"CVSS Score: {nvd_data['base_score']}")
            print(f"CVSS Severity: {nvd_data['base_severity']}")
            print("-" * 40)

            # Sleep for 6 seconds to avoid rate limiting
            time.sleep(6)
        else:
            logging.warning(f"No CVE ID found for index {index}")
            cvss_versions.append(None)
            base_scores.append(None)
            base_severities.append(None)

    # Add the new columns to the DataFrame
    df["cvss_version"] = cvss_versions
    df["base_score"] = base_scores
    df["base_severity"] = base_severities

    # Save the enriched DataFrame back to a Parquet file
    table = pa.Table.from_pandas(df)
    pq.write_table(table, output_file)

    logging.info(f"Enriched data saved to {output_file}")

# Example usage
enrich_parquet_with_nvd_data("output.parquet", "enriched_output.parquet")


2024-08-27 00:18:30,285 - INFO - Processing CVE CVE-2006-6808 (index 0)


CVE: CVE-2006-6808
CVSS Version: 2.0
CVSS Score: 6.8
CVSS Severity: None
----------------------------------------


2024-08-27 00:18:36,684 - INFO - Processing CVE CVE-2007-0233 (index 1)


CVE: CVE-2007-0233
CVSS Version: 2.0
CVSS Score: 7.5
CVSS Severity: None
----------------------------------------


2024-08-27 00:18:43,060 - INFO - Processing CVE CVE-2006-3389 (index 2)


...

2024-08-27 00:21:48,452 - INFO - Processing CVE CVE-2007-5105 (index 39)


CVE: CVE-2007-5105
CVSS Version: 2.0
CVSS Score: 4.3
CVSS Severity: None
----------------------------------------


2024-08-27 00:21:54,832 - INFO - Processing CVE CVE-2006-0985 (index 40)


CVE: CVE-2006-0985
CVSS Version: 2.0
CVSS Score: 4.3
CVSS Severity: None
----------------------------------------


2024-08-27 00:22:01,225 - INFO - Enriched data saved to enriched_output.parquet


In [10]:
import pandas as pd

# Load the Parquet file into a DataFrame
parquet_file_path = "enriched_output.parquet"  # Update this with your Parquet file path
df = pd.read_parquet(parquet_file_path, engine='pyarrow')

# Select only the columns you want to display
columns_to_display = [
    'message', 'cwe', 'cve', 'author', 
    'language', 'application', 'cvss_version', 
    'base_score', 'sarif_file', 'base_severity'
]

# Create a new DataFrame with the selected columns
df_selected = df[columns_to_display]

# Display the first few rows of the selected columns
print("First 40 rows of the selected columns:")
display(df_selected)



First 40 rows of the selected columns:


,message,cwe,cve,author,language,application,cvss_version,base_score,sarif_file,base_severity
0,Improper Neutralization of Input During Web Pa...,CWE-79,CVE-2006-6808,WordPress Foundation,php,cpe:2.3:a:wordpress:wordpress:2.0:-:*:*:*:*:*:*,2.0,6.8,sard_dataset/2015-03-31-wordpress-v2-0/149031-...,MEDIUM
1,Improper Control of Generation of Code ('Code ...,CWE-94,CVE-2007-0233,WordPress Foundation,php,cpe:2.3:a:wordpress:wordpress:2.0:-:*:*:*:*:*:*,2.0,7.5,sard_dataset/2015-03-31-wordpress-v2-0/231383-...,HIGH
2,Externally-Generated Error Message Containing ...,CWE-211,CVE-2006-3389,WordPress Foundation,php,cpe:2.3:a:wordpress:wordpress:2.0:-:*:*:*:*:*:*,2.0,5.0,sard_dataset/2015-03-31-wordpress-v2-0/149040-...,MEDIUM
3,Externally-Generated Error Message Containing ...,CWE-211,CVE-2006-3389,WordPress Foundation,php,cpe:2.3:a:wordpress:wordpress:2.0:-:*:*:*:*:*:*,2.0,5.0,sard_dataset/2015-03-31-wordpress-v2-0/149040-...,MEDIUM
4,Externally-Generated Error Message Containing ...,CWE-211,CVE-2006-3389,WordPress Foundation,php,cpe:2.3:a:wordpress:wordpress:2.0:-:*:*:*:*:*:*,2.0,5.0,sard_dataset/2015-03-31-wordpress-v2-0/149040-...,MEDIUM
5,Improper Control of Generation of Code ('Code ...,CWE-94,CVE-2007-0233,WordPress Foundation,php,cpe:2.3:a:wordpress:wordpress:2.0:-:*:*:*:*:*:*,2.0,7.5,sard_dataset/2015-03-31-wordpress-v2-0/231383-...,HIGH
6,Improper Neutralization of Input During Web Pa...,CWE-79,CVE-2006-1263,WordPress Foundation,php,cpe:2.3:a:wordpress:wordpress:2.0:-:*:*:*:*:*:*,2.0,4.3,sard_dataset/2015-03-31-wordpress-v2-0/149033-...,MEDIUM
7,Improper Neutralization of Input During Web Pa...,CWE-79,CVE-2006-1263,WordPress Foundation,php,cpe:2.3:a:wordpress:wordpress:2.0:-:*:*:*:*:*:*,2.0,4.3,sard_dataset/2015-03-31-wordpress-v2-0/149033-...,MEDIUM
8,Improper Neutralization of Input During Web Pa...,CWE-79,CVE-2006-1263,WordPress Foundation,php,cpe:2.3:a:wordpress:wordpress:2.0:-:*:*:*:*:*:*,2.0,4.3,sard_dataset/2015-03-31-wordpress-v2-0/149033-...,MEDIUM
9,Improper Neutralization of Input During Web Pa...,CWE-79,CVE-2006-1263,WordPress Foundation,php,cpe:2.3:a:wordpress:wordpress:2.0:-:*:*:*:*:*:*,2.0,4.3,sard_dataset/2015-03-31-wordpress-v2-0/149033-...,MEDIUM


In [17]:
import pandas as pd
from IPython.display import display

# Load the Parquet file into a DataFrame
parquet_file_path = "enriched_output.parquet"  # Update this with your Parquet file path
df = pd.read_parquet(parquet_file_path, engine='pyarrow')

# Select only the columns you want to display
columns_to_display = [
    'message', 'cwe', 'cve', 'author', 
    'language', 'application', 'cvss_version', 
    'base_score', 'sarif_file', 'base_severity'
]

# Create a new DataFrame with the selected columns
df_selected = df[columns_to_display]

# Display the first 40 rows of the selected columns with better formatting
styled_df = df_selected.head(40).style.set_table_styles(
    [{
        'selector': 'thead th',
        'props': [('background-color', '#1f77b4'), ('color', 'white')]
    }, {
        'selector': 'tbody tr:nth-child(odd)',
        'props': [('background-color', '#f2f2f2')]
    }, {
        'selector': 'tbody tr:hover',
        'props': [('background-color', '#ffa500')]
    }]
).set_properties(**{
    'border': '1px solid black',
    'padding': '5px',
    'font-family': 'Arial, sans-serif'
})

# Hide the index by using set_table_styles again
styled_df = styled_df.set_table_styles(
    [{'selector': 'row0', 'props': [('display', 'none')]}], overwrite=False
)

# Display the styled DataFrame
display(styled_df)


,message,cwe,cve,author,language,application,cvss_version,base_score,sarif_file,base_severity
0,Improper Neutralization of Input During Web Page Generation ('Cross-site Scripting').,CWE-79,CVE-2006-6808,WordPress Foundation,php,cpe:2.3:a:wordpress:wordpress:2.0:-:*:*:*:*:*:*,2.0,6.800000,"{'$schema': 'https://json.schemastore.org/sarif-2.1.0.json', 'runs': array([{'artifacts': array([{'hashes': {'sha-256': 'd1fecdcc9a980a86df7d788930010fdf8bd848517273b1ce0baf855a758027ef'}, 'length': 4865, 'location': {'uri': 'wp-admin/templates.php'}, 'sourceLanguage': 'php'}], dtype=object), 'properties': {'application': 'cpe:2.3:a:wordpress:wordpress:2.0:-:*:*:*:*:*:*', 'author': 'WordPress Foundation', 'cves': array(['CVE-2006-6808'], dtype=object), 'dependencies': array(['wordpress-v2.0'], dtype=object), 'description': 'CVE-2006-6808', 'id': 149031, 'language': 'php', 'state': 'bad', 'status': 'candidate', 'submissionDate': '2014-08-01', 'type': 'source code', 'version': '1.0.0'}, 'results': array([{'locations': array([{'physicalLocation': {'artifactLocation': {'index': 0, 'uri': 'wp-admin/templates.php'}, 'region': {'endLine': None, 'startLine': 109}}}], dtype=object), 'message': {'text': ""Improper Neutralization of Input During Web Page Generation ('Cross-site Scripting').""}, 'ruleId': 'CWE-79', 'taxa': array([{'id': '79', 'index': 0, 'toolComponent': {'index': 0, 'name': 'CWE'}}], dtype=object)} ], dtype=object), 'taxonomies': array([{'downloadUri': 'https://cwe.mitre.org/data/xml/cwec_v4.6.xml.zip', 'informationUri': 'https://cwe.mitre.org/data/published/cwe_v4.6.pdf', 'isComprehensive': False, 'name': 'CWE', 'organization': 'MITRE', 'shortDescription': {'text': 'The MITRE Common Weakness Enumeration'}, 'taxa': array([{'id': '79', 'name': ""Improper Neutralization of Input During Web Page Generation ('Cross-site Scripting')""}], dtype=object), 'version': '4.6'} ], dtype=object), 'tool': {'driver': {'fullName': 'Software Assurance Reference Dataset Project', 'informationUri': 'https://samate.nist.gov/SARD/', 'name': 'SARD - SAMATE', 'organization': 'NIST', 'supportedTaxonomies': array([{'index': 0, 'name': 'CWE'}], dtype=object), 'version': '5.0.0'}}} ], dtype=object), 'version': '2.1.0'}",MEDIUM
1,Improper Control of Generation of Code ('Code Injection').,CWE-94,CVE-2007-0233,WordPress Foundation,php,cpe:2.3:a:wordpress:wordpress:2.0:-:*:*:*:*:*:*,2.0,7.500000,"{'$schema': 'https://json.schemastore.org/sarif-2.1.0.json', 'runs': array([{'artifacts': array([{'hashes': {'sha-256': '0b38d9ca0f45d0136c4b71ae090317534595887c67e76badc17cd39f82d88a88'}, 'length': 3211, 'location': {'uri': 'wp-trackback.php'}, 'sourceLanguage': 'php'}], dtype=object), 'properties': {'application': 'cpe:2.3:a:wordpress:wordpress:2.0:-:*:*:*:*:*:*', 'author': 'WordPress Foundation', 'cves': array(['CVE-2007-0233'], dtype=object), 'dependencies': array(['wordpress-v2.0'], dtype=object), 'description': 'CVE-2007-0233', 'id': 149029, 'language': 'php', 'state': 'bad', 'status': 'deprecated', 'submissionDate': '2014-08-01', 'type': 'source code', 'version': '1.0.0'}, 'results': array([{'locations': array([{'physicalLocation': {'artifactLocation': {'index': 0, 'uri': 'wp-trackback.php'}, 'region': {'endLine': None, 'startLine': 34}}}], dtype=object), 'message': {'text': ""Improper Control of Generation of Code ('Code Injection').""}, 'ruleId': 'CWE-94', 'taxa': array([{'id': '94', 'index': 0, 'toolComponent': {'index': 0, 'name': 'CWE'}}], dtype=object)} ], dtype=object), 'taxonomies': array([{'downloadUri': 'https://cwe.mitre.org/data/xml/cwec_v4.6.xml.zip', 'informationUri': 'https://cwe.mitre.org/data/published/cwe_v4.6.pdf', 'isComprehensive': False, 'name': 'CWE', 'organization': 'MITRE', 'shortDescription': {'text': 'The MITRE Common Weakness Enumeration'}, 'taxa': array([{'id': '94', 'name': ""Improper Control of Generation of Code ('Code Injection')""}], dtype=object), 'version': '4.6'} ], dtype=object), 'tool': {'driver': {'fullName': 'Software Assurance 

In [16]:
import pandas as pd
import gradio as gr

# Load the Parquet file into a DataFrame
parquet_file_path = "enriched_output.parquet"  # Update this with your Parquet file path
df = pd.read_parquet(parquet_file_path, engine='pyarrow')

# Select only the columns you want to display
columns_to_display = [
    'message', 'cwe', 'cve', 'author', 
    'language', 'application', 'cvss_version', 
    'base_score', 'sarif_file', 'base_severity'
]

# Create a new DataFrame with the selected columns
df_selected = df[columns_to_display].head(40)

# Define a function to display the DataFrame
def display_dataframe():
    return df_selected

# Create a Gradio interface
iface = gr.Interface(
    fn=display_dataframe,
    inputs=None,  # No inputs are needed
    outputs=gr.Dataframe(),  # Use the updated gr.Dataframe class
    live=False,
    title="Vulnerability Data Viewer",
    description="View the first 40 rows of selected vulnerability data."
)

# Launch the Gradio interface
iface.launch()

2024-08-27 02:18:05,477 - INFO - HTTP Request: GET http://127.0.0.1:7861/startup-events "HTTP/1.1 200 OK"
2024-08-27 02:18:05,491 - INFO - HTTP Request: HEAD http://127.0.0.1:7861/ "HTTP/1.1 200 OK"


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


2024-08-27 02:18:05,951 - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
